In [2]:
import gensim
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import pickle
from operator import add
import itertools
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
#import tensorflow as tf
#from sklearn.externals import joblib

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\spradha5\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\spradha5\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#model = gensim.models.Word2Vec.load_word2vec_format(r'C:\Users\Sharvil\GoogleNews-vectors-negative300.bin', binary=True)  
model = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\spradha5\word2vecsharvil\GoogleNews-vectors-negative300.bin.gz', binary=True)

In [4]:
#training dataset
df = pd.read_csv('P1_training_set.csv')
e1 = df['Event 1']
e2 = df['Event 2']
label = df['Label']

In [5]:
def gen_vec(s):
    token = word_tokenize(s)
    words = [word for word in token if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    filtered_words = [w for w in words if not w in stop_words]
    vec = np.zeros(300).tolist()
    for j in filtered_words:
        #model[j]
        #mod_vec = list(itertools.chain.from_iterable(j))
        temp_vec = model[j]
        #vec = list(map(add,vec,temp_vec))
        vec = np.add(vec,temp_vec)
        
    vec = [x/len(filtered_words) for x in vec]
    return vec
        
        

In [6]:
#training vectors
xtrain = []
ytrain = []

for i in range(e1.size):
    s1 = e1.loc[i]
    s2 = e2.loc[i]
    lab = label.loc[i]
    vec1 = gen_vec(s1)
    vec2 = gen_vec(s2)
    vec = vec1+vec2
    
    xtrain.append(vec)
    ytrain.append(lab)
    
xtrain = np.array(xtrain).astype(np.int32)
ytrain = np.array(ytrain).astype(np.int32)



#type(xtrain)
#xtrain.dtype
#ytrain.dtype
#print(len(vec))
#ytrain.shape

KeyError: "word 'Ruxpin' not in vocabulary"

In [8]:
#Random Forrest Training
rfc = RandomForestClassifier(n_jobs=-1, n_estimators=300, max_features='auto')
rfc.fit(xtrain, ytrain)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [15]:
#SVM Training
#stf = LinearSVC(random_state=0, tol=1e-5,dual=True,max_iter=1000,multi_class='ovr')
#stf.fit(xtrain,ytrain)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=0, tol=1e-05,
          verbose=0)

In [18]:
#KNN Training
#kn = KNeighborsClassifier(n_neighbors=9)
#kn.fit(xtrain,ytrain)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                     weights='uniform')

In [9]:
#testing dataset
df2 = pd.read_csv('P1_testing_set.csv')
e1t = df2['Event 1']
e2t = df2['Event 2']
labelt = df2['Label']

In [10]:
#testing
xtest = []
ytest = []

for i in range(e1t.size):
    s1 = e1t.loc[i]
    s2 = e2t.loc[i]
    lab = labelt.loc[i]
    vec1 = gen_vec(s1)
    vec2 = gen_vec(s2)
    vec = vec1+vec2
    
    xtest.append(vec)
    ytest.append(lab)
    
xtest = np.array(xtest).astype(np.int32)
ytest = np.array(ytest).astype(np.int32)

#

#ytest.dtype

#
#xtest.dtype
#ytest.shape
#print(ypredict)
#ypred.shape

In [11]:
#Random Forrest Testing
ypred = rfc.predict(xtest)
print(rfc.score(xtest,ytest)*100)

42.333333333333336


In [16]:
#SVM Testing
#ypred = stf.predict(xtest)
#print(stf.score(xtest,ytest)*100)

42.333333333333336


In [19]:
#KNN Testing
#ypred = kn.predict(xtest)
#print(kn.score(xtest,ytest)*100)

36.888888888888886


In [17]:
#Performance Metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print('\n Confusion Matrix: \n',confusion_matrix(ytest,ypred,labels=[0,1,2]))
print('\n Accuracy: ',accuracy_score(ytest,ypred)*100)
print('\n Precision, Recall Rate, F1-Score: \n',classification_report(ytest,ypred))


 Confusion Matrix: 
 [[  0 187   0]
 [  0 381   0]
 [  0 332   0]]

 Accuracy:  42.333333333333336

 Precision, Recall Rate, F1-Score: 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       187
           1       0.42      1.00      0.59       381
           2       0.00      0.00      0.00       332

    accuracy                           0.42       900
   macro avg       0.14      0.33      0.20       900
weighted avg       0.18      0.42      0.25       900



C:\Users\spradha5\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
